In [1]:
import numpy as np
import theano
import theano.tensor as T

In [2]:
import fbTaskReader as taskReader
reload(taskReader)

# Get the input matrices for the facebook tasks
taskFilePaths = ["/Users/SaahilM/Documents/Princeton/Academics/Thesis/Data/tasks_1-20_v1-2/en/qa1_single-supporting-fact"]
task1 = taskReader.VectorizeTask(taskFilePaths[0])
qaPairs = task1.getTextQuestionPairs()

In [3]:
# theano.config.compute_test_value = 'warn'

# Number of training examples
N = task1.getNumTrainingExamples()
# Number of words in our dictionary
V = task1.getNumWords()
# Dimension to encode our information. They use 20 for independent training
d = 20

print(N,V,d)

# Input X is the collection of sentences, Vx(text_length) matrix
# Input q is our query, a vector of size Vx1
# The actual result is y, a vector of size Vx1
X = T.lmatrix('X')
q = T.lrow('q')
y = T.lvector('y')

X.tag.test_value = np.zeros((2, V), dtype=np.int64)
q.tag.test_value = np.zeros(V, dtype=np.int64)

(1000, 19, 20)


In [4]:
# Create a weight matrix of given size. 
# The matrix is initialized randomly with Gaussian distribution 
# with mean=0 and \sigma=0.1
def initializeWeightMatrix(in_size, out_size):
    return 0.1 * np.random.randn(in_size, out_size)

# Create a bias vector of all zeros of given size
def initializeBiasVector(size):
    return np.zeros(size)

In [7]:
# Initialize all our parameters, given our dimensions.
# A is the first matrix used to embed our input. It has size Vxd
# B is the matrix used to embed the query. It has size Vxd
# C is the next matrix used to embed our input. It has size Vxd
# W is the final matrix. Takes output O and produces result R. It has size dxV
# H is the linear mapping we apply to u_i in each layer, since we have set it
#      up like an RNN
def initializeParams(d, V):
    A = theano.shared(initializeWeightMatrix(V, d))
    B = theano.shared(initializeWeightMatrix(V, d))
    C = theano.shared(initializeWeightMatrix(V, d))
    W = theano.shared(initializeWeightMatrix(d, V))
    H = theano.shared(initializeBiasVector(d)) #H is just a vector
    
#     A = theano.shared(initializeWeightMatrix(d, V))
#     B = theano.shared(initializeWeightMatrix(d, V))
#     C = theano.shared(initializeWeightMatrix(d, V))
#     W = theano.shared(initializeWeightMatrix(V, d))
    return A, B, C, W, H

A, B, C, W, H = initializeParams(d, V)
weightMatrices = [A, B, C, W, H]

In [8]:
# Define the computational step
# Given input matrix X, query q, and weight matrices, we perform a computational step,
# also known as a "hop". Let M be the number of sentences
def hopComputation(X, u, A, B, C, W, H):
    # m_i = Ax_i
    mem_matrix = X.dot(A) #dimension (MxV)x(Vxd) = Mxd
    # p_i = softmax(u^T m_i)
    probs = T.nnet.softmax(mem_matrix.dot(u.T)) #dimension (Mxd)x(dx1) = Mx1
    # C_i = Cx_i
    c_embedded = X.dot(C) #dimension (MxV)x(Vxd) = Mxd
    # output = sum of c_matrix * probs
    o = (probs * c_embedded).sum(axis = 0) #dimension = 1xd
    
    layer_result = H*u + o 
    return result 
    #For each hop computation, we will return the layer_result vector. We will return a 
    # final result multiplied by vector W at the very end